In [ ]:
from simsopt.geo import SurfaceRZFourier
import numpy as np

# File for the desired boundary magnetic surface:
filename = "tests/test_files/input.LandremanPaul2021_QA" # "tests/test_files/input.LandremanPaul2021_QH_reactorScale_lowres"

# Initialize the boundary magnetic surface:
s = SurfaceRZFourier.from_vmec_input(filename, range="half period")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def plot_modes(s:SurfaceRZFourier):
  extent = (-s.rc.shape[1]/2,s.rc.shape[1]/2,s.rc.shape[0]+0.5,-0.5)

  plt.subplot(211)
  plt.imshow(s.rc*1e3, norm="symlog", extent=extent)
  plt.colorbar()
  plt.title("Rcos")
  plt.ylabel("n")
  plt.subplot(212, sharex=plt.gca())
  plt.imshow(s.zs*1e3, norm="symlog", extent=extent)
  plt.colorbar()
  plt.title("Zsin")
  plt.ylabel("n")
  plt.xlabel("m")


## Vary B.n

In [ ]:

# Create the initial coils:
base_curves = create_equally_spaced_curves(ncoils, s.nfp, stellsym=True, R0=R0, R1=R1, order=order)
base_currents = [Current(1e5) for i in range(ncoils)]
# Since the target field is zero, one possible solution is just to set all
# currents to 0. To avoid the minimizer finding that solution, we fix one
# of the currents:
base_currents[0].fix_all()

coils = coils_via_symmetries(base_curves, base_currents, s.nfp, True)
bs = BiotSavart(coils)
bs.set_points(s.gamma().reshape((-1, 3)))

## Vary the geometry 

In [ ]:
fourier_limits = list(range(6,0, -1))

for m,n in zip(reversed(s.m), reversed(s.n)):
  #s.set_rc(m,n,0.0)
  s.set_zs(m,n,0.0)
  if (n<3):
    plot_modes(s)
    s.plot()
    plt.show()

In [ ]:
from examples.Simple import stage_two_optimization_minimal
coilss = []

fourier_limits = list(range(6,1, -1))

for limit in fourier_limits:
  mask = np.zeros_like(s.rc)
  for m,n in zip(s.m, s.n):
    if(abs(m)>=limit or abs(n)>=limit):
      s.set_rc(m,n,0.0)
      s.set_zs(m,n,0.0)

  #plt.matshow(s.rc*1e3, norm="symlog")
  #plt.colorbar()
  #plt.title(f"Rcos {limit}")
  #plt.show()

  coils = stage_two_optimization_minimal.optimize(s)
  coilss.append(coils)


In [ ]:
from simsopt.geo import plot, LpCurveCurvature, fix_matplotlib_3d

# Power of the norm, e.g. L2
p = 2

for coils, limit in zip(coilss, fourier_limits):
  curvatures = [LpCurveCurvature(coil.curve, p).J() for coil in coils]
  plt.scatter([limit]*len(curvatures), curvatures)
plt.ylabel("L2 norm of Curvature")
plt.xlabel("Cutoff for |m|,|n| modes of B.n")
plt.legend([f"$|m|,|n|<{limit}$" for limit in fourier_limits])
plt.show()

for coil_i in range(len(coilss[0])):
  ax = plt.figure(figsize=(16,10)).add_subplot(projection='3d')
  for coils in coilss:
    coils[coil_i].plot(ax=ax, show=False)
  plt.legend([f"$|m|,|n|<{limit}$" for limit in fourier_limits])
  ax.set_prop_cycle(None) 
  fix_matplotlib_3d(ax)
  pass